

This notebook performs the Extract, Transform, and Load (ETL) process
for a PayPal-like transaction dataset used in a Business Intelligence project.


In [40]:
import pandas as pd

df = pd.read_csv("../data_raw/paypal_transactions_raw.csv")
df.head()



,transaction_id,transaction_date,transaction_amount,transaction_fee,transaction_status,fraud_flag,customer_id,customer_type,merchant_id,merchant_category,payment_method,country
0,1,2023-04-13,446.76,4.10,Success,0,4640,Existing,397,Gaming,Credit Card,USA
1,2,2023-12-15,606.62,1.10,Success,0,3605,Existing,532,Gaming,Debit Card,ES
2,3,2023-09-28,337.18,1.60,Success,0,6418,New,599,Gaming,Credit Card,TN
3,4,2023-04-17,133.42,4.34,Success,0,3002,Existing,609,Travel,Debit Card,TN
4,5,2023-03-13,390.16,2.85,Failed,0,6629,Existing,771,Gaming,Credit Card,USA


In [41]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   transaction_id      30000 non-null  int64  
 1   transaction_date    30000 non-null  object 
 2   transaction_amount  30000 non-null  float64
 3   transaction_fee     30000 non-null  float64
 4   transaction_status  30000 non-null  object 
 5   fraud_flag          30000 non-null  int64  
 6   customer_id         30000 non-null  int64  
 7   customer_type       30000 non-null  object 
 8   merchant_id         30000 non-null  int64  
 9   merchant_category   30000 non-null  object 
 10  payment_method      30000 non-null  object 
 11  country             30000 non-null  object 
dtypes: float64(2), int64(4), object(6)
memory usage: 2.7+ MB


In [42]:
df = df.drop_duplicates(subset="transaction_id")


In [43]:
df["transaction_fee"] = df["transaction_fee"].fillna(0)
df["fraud_flag"] = df["fraud_flag"].fillna(0)


In [44]:
df["transaction_date"] = pd.to_datetime(df["transaction_date"])
df["transaction_amount"] = df["transaction_amount"].astype(float)


In [45]:
df["transaction_status"] = df["transaction_status"].str.lower()
df["country"] = df["country"].str.upper()


In [46]:
q1 = df["transaction_amount"].quantile(0.25)
q3 = df["transaction_amount"].quantile(0.75)
iqr = q3 - q1

df = df[
    (df["transaction_amount"] >= q1 - 1.5 * iqr) &
    (df["transaction_amount"] <= q3 + 1.5 * iqr)
]


In [47]:
df["year"] = df["transaction_date"].dt.year
df["month"] = df["transaction_date"].dt.month
df["hour"] = df["transaction_date"].dt.hour


In [48]:
df.to_csv("../data_cleaned/paypal_transactions_clean.csv", index=False)


In [49]:
df.head()


,transaction_id,transaction_date,transaction_amount,transaction_fee,transaction_status,fraud_flag,customer_id,customer_type,merchant_id,merchant_category,payment_method,country,year,month,hour
0,1,2023-04-13,446.76,4.10,success,0,4640,Existing,397,Gaming,Credit Card,USA,2023,4,0
1,2,2023-12-15,606.62,1.10,success,0,3605,Existing,532,Gaming,Debit Card,ES,2023,12,0
2,3,2023-09-28,337.18,1.60,success,0,6418,New,599,Gaming,Credit Card,TN,2023,9,0
3,4,2023-04-17,133.42,4.34,success,0,3002,Existing,609,Travel,Debit Card,TN,2023,4,0
4,5,2023-03-13,390.16,2.85,failed,0,6629,Existing,771,Gaming,Credit Card,USA,2023,3,0
